In [11]:
from selenium import webdriver
from time import sleep

driver = webdriver.Chrome('/Users/iliasmiraoui/Desktop/chromedriver')

In [140]:

artist= "Coldplay"
title="Hymn+for+the+weekend"
url = "https://www.riaa.com/gold-platinum/?tab_active=default-award&ar="+artist+"&ti="+title+"&lab=&genre=&format=&date_option=release&from=&to=&award=&type=&category=&adv=SEARCH#search_section"

In [144]:
driver.get(url)

In [145]:
driver.find_element_by_xpath("//*/td[6]/div/a").click()
sleep(1)
driver.find_elements_by_xpath('//*/div/table/tbody/tr[2]')[1].text

IndexError: list index out of range